# Performs double-counting removal

In [1]:
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
mapping_technologies = pd.read_csv('energyscope_data/mapping_linked.csv') # mapping between the esm technologies and LCI datasets
mapping_esm_flows_to_CPC = pd.read_csv('energyscope_data/mapping_esm_flows_to_CPC.csv') # mapping between the esm flows and CPC categories
model = pd.read_csv('energyscope_data/model.csv') # input output definitions of the esm technologies 
technology_compositions = pd.read_csv('energyscope_data/technology_compositions.csv') # compositions of the esm technologies
technology_specifics = pd.read_csv('energyscope_data/technology_specifics.csv') # specific requirements for the esm technologies

In [3]:
bd.projects.set_current("ei3.8-mescal")

In [4]:
db = concatenate_databases(list(mapping_technologies.Database.unique()))

In [5]:
new_db_name = 'energyscope_QC_2020'

## New database without foreground regionalization

### Database creation

In [6]:
create_esm_database(
    mapping=mapping_technologies, 
    model=model, 
    tech_specifics=technology_specifics, 
    technology_compositions=technology_compositions, 
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC, 
    main_database=db,
    esm_db_name=new_db_name,
    results_path_file='energyscope_data/results/'
)

### Verification

In [7]:
new_db = load_extract_db(new_db_name, create_pickle=False)

In [14]:
res = []
for act in new_db:
    if 'Subject to double-counting removal (market)' in act['comment']: # we only look at operation LCI datasets that performed double-counting removal
        for exc in get_technosphere_flows(act):
            if ('maintenance' in exc['name']) | ('treatment' in exc['name']): # we keep maintenance and treatment in the operation LCI (proportional to use)
                pass
            elif ('EV charger' in exc['name']) | ('airport' in exc['name']): # these entries are proportional to the use phase also
                pass
            else:
                if (exc['amount'] != 0 ) & (exc['unit'] == 'unit'):
                    print(f"Activity: {act['name']}")
                    print(f"Exchange: {exc['name']}")
                    act_res = [a for a in db if a['code'] == exc['code']][0]
                    res.append(act_res['reference product'])
                else:
                    pass
    else:
        pass

In [15]:
list(set(res))

## New database with foreground regionalization

### Database creation

In [6]:
my_ranking = [
    'CA-QC',
    'CA',
    'CA-ON',
    'CA-AB',
    'CA-BC',
    'CA-MB',
    'CA-NB',
    'CA-NF',
    'CA-NS',
    'CA-NT',
    'CA-NU',
    'CA-PE',
    'CAZ',
    'RNA',
    'US',
    'USA',
    'GLO',
    'RoW',
    'RER',
]

In [7]:
mismatch_regions = ['GLO', 'RoW', 'RER', 'CH'] # insufficient match within ecoinvent

In [8]:
create_esm_database(
    mapping=mapping_technologies, 
    model=model, 
    tech_specifics=technology_specifics, 
    technology_compositions=technology_compositions, 
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC, 
    main_database=db,
    esm_db_name=new_db_name+"_reg",
    results_path_file='energyscope_data/results/',
    regionalize_foregrounds=True,
    mismatch_regions=mismatch_regions,
    target_region='CA-QC',
    locations_ranking=my_ranking
)